In [3]:
import httpx
from pydantic import BaseModel
import json

In [4]:

class KBResponse(BaseModel):
    query: str
    text: str
    score: float
    id: str
    tag: str
    name: str

In [ ]:
# with open("/Users/steffen/sim_company/chemical/raw/assets.json", "r") as f:
#     data = json.load(f)

In [ ]:
# mapping = [
#     ("temp vor Rückstandskühler", "TI-T0022"),
#     ("temp nach Rückstandskühler", "TI-T0026"),
#     ("stand nach Rückstandskühler", "LI-L0009"),
#     ("temp nach Destillatkühler a", "TI-T0027"),
#     ("temp nach Destillatkühler b", "TI-T0028"),
#     ("stand nach Destillatkühler a", "LI-L0010"),
#     ("stand nach Destillatkühler b", "LI-L0008"),
#     ("druck nach Destillatkühler a", "PI-P0020"),
#     ("druck nach Destillatkühler b", "PI-P0019"),
#     ("menge an zufluss in tanklager", "FQI-F0024"),
#     ("dampfdruck in der destillation", "PDI-P0018"),
#     ("obere Temperatur in der destillation", "TRC-T0029"),
#     ("mittlere Temperatur in der destillation", "TRC-T0031"),
#     ("untere Temperatur in der destillation", "TRC-T0032"),
#     ("menge in der destillation", "LRC-L0011"),
#     ("zuflussmenge zur destillation", "FRC-F0025"),
#     ("zuflusstemperatur zur destillation", "TI-T0030"),
#     ("temp Kühlwasser-Rücklauf von der Destillation", "TI-T0023"),
#     ("menge Kühlwasser-Rücklauf von der Destillation", "FI-F0023"),
#     ("temp Kühlwasser-Rücklauf von Kondensator", "TI-T0023"),
#     ("menge Kühlwasser-Rücklauf von Kondensator", "FI-F0023"),
#     ("temp Kühlgas-Rücklauf von der Destillation", "TI-T0025"),
#     ("temp Kühlgas-Rücklauf von Gaskühler", "TI-T0025"),
# ]

In [ ]:
# raw = []
# for desc, tag in mapping:
#     response = httpx.get(f"http://localhost:5050/v1/id_from_name/{tag}")
#     _id = response.json()
#     raw.append({"description": desc, "name": tag, "tag": tag, "id": _id})

In [ ]:
# with open("/Users/steffen/sim_company/chemical/raw/knowledge_base.json", "w") as f:
#     json.dump(raw,f)

In [ ]:
test = httpx.get("http://localhost:5051/v1/embedding/test")

In [ ]:
n_emb = len(test.json()["embedding"])

## Data ingestion

In [ ]:
with open("/Users/steffen/sim_company/chemical/raw/knowledge_base.json", "r") as f:
    data = json.load(f)

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(host="localhost", port=6333)


In [ ]:
from qdrant_client.models import VectorParams, Distance

# client.delete_collection(collection_name="chemical")


if not client.collection_exists("chemical"):
   client.create_collection(
      collection_name="chemical",
      vectors_config=VectorParams(size=n_emb, distance=Distance.COSINE),
   )


In [ ]:
# import numpy as np
# from qdrant_client.models import PointStruct

# for idx, _data in enumerate(data):

#     response = httpx.get(f"http://localhost:5051/v1/embedding/{_data["description"]}")
#     embedding = response.json()["embedding"]
    
#     client.upsert(
#         collection_name="chemical",
#         points=[
#             models.PointStruct(
#                 id=_data["id"],
#                 payload=_data,
#                 vector=embedding,
#             ),
#         ],
#     )



## Data retrieval

In [ ]:
# query_text = "What is the temperature?"

# response = httpx.get(f"http://localhost:5051/v1/embedding/", params={"text": query_text})
# embedding = response.json()["embedding"]

In [ ]:

# response = httpx.post(f"http://localhost:5051/v1/search/", json={"embedding": embedding})
# response = response.json()

In [ ]:
# scores = []

# for p in response["results"]:
#     r = httpx.get(f"http://localhost:5051/v1/ranking/", params={"text": p["description"], "query": query_text})
#     score = r.json()
#     p.pop("score", None)
#     data = KBResponse(**score, **p)
#     scores.append(data)

# scores = sorted(scores, key=lambda x: -x.score)

In [ ]:
# scores[0]

In [5]:
def get_candidates(query):
    scores = []

    response = httpx.get(f"http://localhost:5051/v1/embedding/", params={"text": query})
    embedding = response.json()["embedding"]

    response = httpx.post(f"http://localhost:5051/v1/search/", json={"embedding": embedding})
    response = response.json()
    
    for p in response["results"]:
        r = httpx.get(f"http://localhost:5051/v1/ranking/", params={"text": p["description"], "query": query})
        score = r.json()
        p.pop("score", None)
        data = KBResponse(**score, **p)
        scores.append(data)
    
    scores = sorted(scores, key=lambda x: -x.score)

    return scores

In [6]:
queries = ["What is the daily maximum value of PI-P0017 in April 2025?",
           "How much was produced in the first two weeks of 2025?",
            "How much was the total production in the first two weeks of 2025?",
            "How much was the total production in the distillation first two weeks of 2025?",
            "Can you compare PI-P0017 and PI-P0016 for the first 10 days in 2025?",
            "What is the last pressure in the distillation?",
            "What is the leven in tank b?",
            "can you plot me the tempurate of the distillation cooler A for the last two weeks?",
            "What is the temperature of the water tank?",
            "what is the current pressure in the tank storage a?",
           "how much distillate is flown in the storage in the last 4 hours?",
            "what is the current temperature at top of the distillation column?"

]

In [7]:
i = 0

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

What is the daily maximum value of PI-P0017 in April 2025?
-10.926222801208496 total production rate of the distillation


In [8]:
i = 1

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

How much was produced in the first two weeks of 2025?
-10.114917755126953 total production rate of the distillation


In [9]:
i = 2

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

How much was the total production in the first two weeks of 2025?
-8.1950101852417 total production rate of the distillation


In [10]:
i = 3

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

How much was the total production in the distillation first two weeks of 2025?
-0.8265450596809387 total production rate of the distillation


In [11]:
i = 4

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

Can you compare PI-P0017 and PI-P0016 for the first 10 days in 2025?
-9.457514762878418 total production rate of the distillation


In [12]:
i = 5

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

What is the last pressure in the distillation?
1.3120077848434448 vapor pressure in the distillations column


In [13]:
i = 6

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

What is the leven in tank b?
-4.3909406661987305 destillate level before the tank storage at tank b


In [14]:
i = 7

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)
print(scores[1].score, scores[1].text)
print(scores[2].score, scores[2].text)

# destillate temperature after the distillation cooler at tank a

can you plot me the tempurate of the distillation cooler A for the last two weeks?
-4.500114440917969 temperature of the distillate flow towards the distillation column
-5.038633823394775 temperature in the upper part of the distillation column
-5.061733722686768 temperature in the middle part of the distillation column


In [15]:
i = 8

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

What is the temperature of the water tank?
-0.13996659219264984 measures the cooling water tank temperature after the residue cooler


In [16]:
i = 9

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

what is the current pressure in the tank storage a?
3.474311351776123 destillate pressure before the tank storage at tank a


In [17]:
i = 10

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)
print(scores[1].score, scores[1].text)
print(scores[2].score, scores[2].text)

# amount of distillate flow towards tank storage

how much distillate is flown in the storage in the last 4 hours?
-5.278166770935059 amount of distillate flow towards the distillation column
-5.958638668060303 level of distillate in the distillation column
-7.514389991760254 total production rate of the distillation


In [18]:
i = 11

scores = get_candidates(queries[i])

print(queries[i])
print(scores[0].score, scores[0].text)

what is the current temperature at top of the distillation column?
1.4172132015228271 temperature in the upper part of the distillation column
